In [1]:
###转换tfrecord
import os
import gc
import cv2
import numpy as np
import datetime
def get_time():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


image_dir = "input/images"
train_dict = {}
val_dict = {}
test_dict = {}
label_list=[]

for label_dir in os.listdir(image_dir):
    class_path = os.path.join(image_dir,label_dir)
    filenames_path = [ image_dir+'/'+label_dir+'/'+filename for filename  in os.listdir(class_path)]
    train_dict[label_dir]=filenames_path[:int(0.8*len(filenames_path))]
    val_dict[label_dir]=filenames_path[int(0.8*len(filenames_path)):int(0.9*len(filenames_path))]
    test_dict[label_dir]=filenames_path[int(0.9*len(filenames_path)):]
    label_list.append(label_dir)

    

In [5]:
for i,j in enumerate(label_list):
    print("\"%s\" : \"%s\"," %(i,j))

"0" : "Abyssinian",
"1" : "American Bobtail",
"2" : "American Curl",
"3" : "American Shorthair",
"4" : "American Wirehair",
"5" : "Applehead Siamese",
"6" : "Balinese",
"7" : "Bengal",
"8" : "Birman",
"9" : "Bombay",
"10" : "British Shorthair",
"11" : "Burmese",
"12" : "Burmilla",
"13" : "Calico",
"14" : "Canadian Hairless",
"15" : "Chartreux",
"16" : "Chausie",
"17" : "Chinchilla",
"18" : "Cornish Rex",
"19" : "Cymric",
"20" : "Devon Rex",
"21" : "Dilute Calico",
"22" : "Dilute Tortoiseshell",
"23" : "Domestic Long Hair",
"24" : "Domestic Medium Hair",
"25" : "Domestic Short Hair",
"26" : "Egyptian Mau",
"27" : "Exotic Shorthair",
"28" : "Extra-Toes Cat - Hemingway Polydactyl",
"29" : "Havana",
"30" : "Himalayan",
"31" : "Japanese Bobtail",
"32" : "Javanese",
"33" : "Korat",
"34" : "LaPerm",
"35" : "Maine Coon",
"36" : "Manx",
"37" : "Munchkin",
"38" : "Nebelung",
"39" : "Norwegian Forest Cat",
"40" : "Ocicat",
"41" : "Oriental Long Hair",
"42" : "Oriental Short Hair",
"43" : "Orienta

In [ ]:
label_imgs,des_dir=train_dict,image_tf_dir+"/inception_data/image_train_tfrcds"

flag = 0
writer = None
model_path='model/frozen_model/inception/classify_image_graph_def.pb'
unfreez_tensor={'input':'DecodeJpeg/contents:0',
                        'output':'pool_3:0'}
with tf.Graph().as_default(),tf.Session().as_default() as sess:
    tensors=unfreez(model_path,unfreez_tensor)
    for label,images in label_imgs.items():
        for image in images:

In [7]:
###转换tfrecord
import os
import tensorflow as tf 
import gc
import cv2
import numpy as np
image_dir = "input/images"
train_dict = {}
val_dict = {}
test_dict = {}
label_list=[]

for label_dir in os.listdir(image_dir):
    class_path = os.path.join(image_dir,label_dir)
    filenames_path = [ image_dir+'/'+label_dir+'/'+filename for filename  in os.listdir(class_path)]
    train_dict[label_dir]=filenames_path[:int(0.8*len(filenames_path))]
    val_dict[label_dir]=filenames_path[int(0.8*len(filenames_path)):int(0.9*len(filenames_path))]
    test_dict[label_dir]=filenames_path[int(0.9*len(filenames_path)):]
    label_list.append(label_dir)

    
def unfreez(model_path,unfreez_tensor_dict):
    with tf.gfile.GFile(name=model_path, mode='rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
        g = tf.get_default_graph()
        tensors = {t: g.get_tensor_by_name(unfreez_tensor_dict[t]) for t in unfreez_tensor_dict}
        return tensors
    

def to_inception_tfrecords(label_imgs,des_dir):
    flag = 0
    writer = None
    model_path='model/frozen_model/inception/classify_image_graph_def.pb'
    unfreez_tensor={'input':'DecodeJpeg/contents:0',
                            'output':'pool_3:0'}
    with tf.Graph().as_default(),tf.Session().as_default() as sess:
        tensors=unfreez(model_path,unfreez_tensor)
        for label,images in label_imgs.items():
            for image in images:
                if flag%1000 == 0 :
                    print("flag is :",flag)
                    if writer:
                        writer.flush()
                        writer.close()
                    tf_path = os.path.join(des_dir,"%s_%s.tfrecords" %(flag+1,flag+1000))
                    print(tf_path)
                    writer = tf.python_io.TFRecordWriter(tf_path)
                flag +=1
                try:
                    img_raw =open(image,'rb').read()
                    if  img_raw is None:
                        print("can't open picture: ",image)
                        continue
                except:
                    print("can't recognize picture: ",image)
                    continue
                inception_out=sess.run(tensors['output'],{tensors['input']:img_raw}).reshape(-1).tobytes()
                label_ = label.encode("utf-8") ## one hot encoder ?

                example = tf.train.Example(features=tf.train.Features(feature={
                                                                   'label':tf.train.Feature(bytes_list=tf.train.BytesList(value=[label_])),
                                                                  'img_incept':tf.train.Feature(bytes_list=tf.train.BytesList(value=[inception_out]))
                                                                                        } )
                                                              )
                writer.write(example.SerializeToString())

                del img_raw
                del inception_out
                del example
                gc.collect()
                writer.flush()
            break
                
    writer.close()
    print(des_dir,' written complete!')

    
image_tf_dir = "input/tfrecord/inception_data/"
to_inception_tfrecords(train_dict,image_tf_dir+"image_train_tfrcds")
to_inception_tfrecords(val_dict,image_tf_dir+"image_val_tfrcds")
to_inception_tfrecords(test_dict,image_tf_dir+"image_test_tfrcds")

flag is : 0
input/tfrecord/inception_data/image_train_tfrcds\1_1000.tfrecords
input/tfrecord/inception_data/image_train_tfrcds  written complete!
flag is : 0
input/tfrecord/inception_data/image_val_tfrcds\1_1000.tfrecords
input/tfrecord/inception_data/image_val_tfrcds  written complete!
flag is : 0
input/tfrecord/inception_data/image_test_tfrcds\1_1000.tfrecords
input/tfrecord/inception_data/image_test_tfrcds  written complete!


In [4]:
open("input/images/Abyssinian/14092337_250.jpg",'rb').read()

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00;CREATOR: gd-jpeg v1.0 (using IJG JPEG v62), quality = 80\n\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc2\x00\x11\x08\x01\xc9\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x01\x02\x03\x05\x06\x00\x07\x08\xff\xc4\x00\x17\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\xff\xda\x00\x0c\x03\x01\x00\x02\x10\x03\x10\x00\x00\x01\xf5H\xb3d\xa1\x99k\xec\xa5&\xe3\xcbi#\xde\xba\x9a\xe7S\x91xEE9\x15\x05N\xe3\xbb\xb8\xee\xee\x13\x95\x0e\xee\xe19PNT\x11\x15\x04k\x91\x1a\x8a\x94\x9d\xdd\t\xdd\xc6q\x1c\xecn*\xe7Dvor%`=\xcb\xc7\tOc\x